# Clustering metrics - alternatives to the _elbow method_
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536

Кластеризация — важная часть конвейера машинного обучения для бизнеса или научных предприятий, использующих науку о данных. Как следует из названия, это помогает идентифицировать группы тесно связанных (по некоторой мере расстояния) точек данных в блоке данных, которые в противном случае было бы трудно понять.

Популярный метод, такой как кластеризация k-средних, кажется, не дает полностью удовлетворительного ответа, когда мы задаем основной вопрос:
> **"Как нам узнать фактическое количество кластеров для начала?"**

Этот вопрос является критически важным, поскольку процесс кластеризации часто предшествует дальнейшей обработке данных отдельных кластеров, и поэтому объем вычислительных ресурсов может зависеть от этого измерения.

В случае с бизнес-аналитикой последствия могут быть хуже. Для такой аналитики часто делается кластеризация с целью сегментации рынка. Поэтому легко предположить, что в зависимости от количества кластеров для решения проблемы будет выделен соответствующий маркетинговый персонал. Следовательно, неправильная оценка количества кластеров может привести к неоптимальному распределению ценных ресурсов.

Для метода кластеризации k-средних наиболее распространенным подходом к ответу на этот вопрос является так называемый метод локтя. Он включает в себя запуск алгоритма несколько раз в цикле с увеличением числа вариантов выбора кластера, а затем построение оценки кластеризации в зависимости от количества кластеров.

В этой записной книжке мы показываем, какую метрику использовать для визуализации и определения оптимального количества кластеров намного лучше, чем обычный метод «локтя».

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Create synthetic data using Scikit learn `make_blob` method

- Number of features: 4
- Number of clusters: 5
- Number of samples: 200

In [ ]:
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs

In [ ]:
n_features = 4
n_cluster = 5
cluster_std = 1.2
n_samples = 200

In [ ]:
data1 = make_blobs(n_samples=n_samples,n_features=n_features,centers=n_cluster,cluster_std=cluster_std)

In [ ]:
d1 = data1[0]

In [ ]:
df1=pd.DataFrame(data=d1,columns=['Feature_'+str(i) for i in range(1,n_features+1)])
df1.head()

In [ ]:
from itertools import combinations

In [ ]:
lst_vars=list(combinations(df1.columns,2))

In [ ]:
len(lst_vars)

In [ ]:
plt.figure(figsize=(15,8))
for i in range(1,7):
    plt.subplot(2,3,i)
    dim1=lst_vars[i-1][0]
    dim2=lst_vars[i-1][1]
    plt.scatter(df1[dim1],df1[dim2],c=data1[1],edgecolor='k',s=150)
    plt.xlabel(f"{dim1}",fontsize=13)
    plt.ylabel(f"{dim2}",fontsize=13)

### How are the classes separated (boxplots)

In [ ]:
plt.figure(figsize=(16,14))
for i,c in enumerate(df1.columns):
    plt.subplot(3,2,i+1)
    sns.boxplot(y=df1[c],x=data1[1])
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel("Class",fontsize=15)
    plt.ylabel(c,fontsize=15)
    #plt.show()

## k-means clustering

In [ ]:
from sklearn.cluster import KMeans

### Unlabled data

In [ ]:
X=df1

In [ ]:
X.head()

In [ ]:
y=data1[1]

### Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_scaled=scaler.fit_transform(X)

### Metrics

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score

### Running k-means and computing inter-cluster distance score for various *k* values

In [ ]:
k_inertia = []
ks = range(1,11)

for k in ks:
    clf_kmeans = KMeans(n_clusters=k)
    clusters_kmeans = clf_kmeans.fit_predict(X_scaled, )
    k_inertia.append(clf_kmeans.inertia_)

In [ ]:
X_scaled

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
k_inertia = []
for i in range(2,12):
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)

    k_inertia.append(km.inertia_)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(X_scaled)))
    km_scores.append(-km.score(X_scaled))
    
    silhouette = silhouette_score(X_scaled,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(X_scaled,preds)
    db_score.append(db)
    print("Davies Bouldin score for number of cluster(s) {}: {}".format(i,db))
    
    v_measure = v_measure_score(y,preds)
    vmeasure_score.append(v_measure)
    print("V-measure score for number of cluster(s) {}: {}".format(i,v_measure))
    print("-"*100)



In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of k_inertia\n",fontsize=16)
plt.scatter(x=[i for i in range(2,12)],y=k_inertia,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,12)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,12)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,12)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,12)],y=vmeasure_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("V-measure score")
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,12)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=15)
plt.xticks([i for i in range(2,12)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,12)],y=db_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Davies-Bouldin score")
plt.show()

## Expectation-maximization (Gaussian Mixture Model)

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm_bic= []
gm_score=[]
for i in range(2,12):
    gm = GaussianMixture(n_components=i,n_init=10,tol=1e-3,max_iter=1000).fit(X_scaled)
    print("BIC for number of cluster(s) {}: {}".format(i,gm.bic(X_scaled)))
    print("Log-likelihood score for number of cluster(s) {}: {}".format(i,gm.score(X_scaled)))
    print("-"*100)
    gm_bic.append(-gm.bic(X_scaled))
    gm_score.append(gm.score(X_scaled))

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The Gaussian Mixture model BIC \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,12)],y=np.log(gm_bic),s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Log of Gaussian mixture BIC score",fontsize=15)
plt.xticks([i for i in range(2,12)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,12)],y=gm_score,s=150,edgecolor='k')
plt.show()

---

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
N_FEATURES=8
N_CLASSES=10
X,Y=make_blobs(n_samples=2000, 
                      n_features=N_FEATURES, 
                      centers=np.random.uniform(0,9,size=(N_CLASSES,N_FEATURES)), 
                      random_state=42, )
TOTAL_SAMPLES=X.shape[0]
import matplotlib.pyplot as pp
pp.scatter(X[:,0],X[:,2],c=Y);

In [ ]:
X

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# используем метод fit_transform вместо fit, т.к. класс TSNE не использует метод transform
X_tsne = tsne.fit_transform(X)


In [ ]:
X_tsne.shape

In [ ]:
TOTAL_SAMPLES=X.shape[0]
import matplotlib.pyplot as pp
pp.scatter(X_tsne[:,0],X_tsne[:,1],c=Y);
plt.xlabel("t-SNE признак 0")
plt.xlabel("t-SNE признак 1")

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score
#X_scaled=scaler.fit_transform(X)
X_scaled = X
for i in range(2,15):
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(X_scaled)))
    km_scores.append(-km.score(X_scaled))
    
    silhouette = silhouette_score(X_scaled,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(X_scaled,preds)
    db_score.append(db)
    print("Davies Bouldin score for number of cluster(s) {}: {}".format(i,db))
    
    v_measure = v_measure_score(Y,preds)
    vmeasure_score.append(v_measure)
    print("V-measure score for number of cluster(s) {}: {}".format(i,v_measure))
    print("-"*100)

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,15)],y=vmeasure_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("V-measure score")
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,15)],y=db_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Davies-Bouldin score")
plt.show()

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
from math import nan


TRAIN_CSV_PATH = 'titanic_train.csv'
TEST_CSV_PATH = 'titanic_test.csv'

In [ ]:
def process_data(df:pd.DataFrame):
 
    # Для признака "Cabin" - часть пропущенных значений заменить совпадающими по номеру билета (по одному билету может быть зарегистрирована группа пассажиров)
    df['Cabin'] = df.groupby(['Ticket'])['Cabin'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))

    # Выделить из признака "Cabin"(номер каюты) палубу "Deck"
    df.insert(10,'Deck', df['Cabin'].str.extract(r'(\D+)', expand = True)[0])
    # Исключить из признака ошибочные буквы, например "f e"  или "f g" - при этом использовать последнюю букву 
    df['Deck'] = df['Deck'].str[-1:]

    # Выделить из признака Name новый признак Title(титул-приставка к имени пассажира)
    df.insert(2,'Title', df['Name'].str.extract(r' (\w+)\.', expand = True)[0] )
    df.loc[df['Title'].isin(['capt','col','major']), 'Title'] = 'military'
    df.loc[df['Title'].isin(['lady','donа','countess']), 'Title'] = 'lady'
    df.loc[df['Title'].isin(['sir','don','jonkheer']), 'Title'] = 'sir'
    df.loc[df['Title'].isin(['ms','mlle']), 'Title'] = 'miss'
    df.loc[df['Title'].isin(['mme']), 'Title'] = 'mrs'

    # Для признака "Age" - заменить пропущенные значения медианным значением группы по признаку 'Title'(титул)
    df['Age'] = df.groupby(['Title'])['Age'].transform(lambda x: x.fillna(x.median() if x.count()>0 else nan))
    # ВЫделить признак возрастной категории - до 1 года, до 10 лет, и т.д.
    df.insert(6,'Age_category', df['Age'])
    df['Age_category'] = df['Age_category'].transform(lambda x: round(x,-1)+10 if x > 1 else 1)

    #df.insert(9,'Relatives', df['SibSp'] + df['Parch'] +1)

    # Выделить признак количества пассажиров зарегистрированных по одному билету
    df.insert(10,'Persons_per_ticket', df.groupby('Ticket')['Fare'].transform('count'))
    # Заменить значения 0 на 1
    df['Persons_per_ticket'] = df['Persons_per_ticket'].apply(lambda x: 1 if x == 0 else x)

    # Выделить признак стоимости одного проездного места в билете
    df.insert(11,'Price_per_person', (df['Fare'] / df['Persons_per_ticket']).round(decimals=2))
    df['Price_per_person'] = df.groupby(['Pclass'])['Price_per_person'].transform(lambda x: x.fillna(x.median() if x.count()>0 else 0))

    # Для признака "Deck" часть пропущенных значений заменить совпадающими по классу и стоимости места
    df['Deck'] = df.groupby(['Pclass','Price_per_person'])['Deck'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
    # Значения не соответсвующие ни одной из палуб корабля отнести к категории Unknown
    df['Deck'] = df['Deck'].apply(lambda x: 'unknown' if x not in ['a','b','c','d','e','f','g'] else x)
    # Оставшиеся пропущенные значения отнести к категории Unknown
    df['Deck'].fillna('unknown', inplace=True)

    # Для признака "Embarked" - заменить пропущенные значения наиболее частым значением данного признака
    df['Embarked'].fillna(df.mode()['Embarked'][0], inplace=True)

    return df

In [ ]:
df_raw = pd.read_csv(TRAIN_CSV_PATH, sep=',')

# Привести все строковые значения к единому регистру, 
df_raw = df_raw.apply(lambda x: x.astype(str).str.lower() if x.dtype.name=='object' else x)
# Удалить столбец 'PassengerId'
df_raw.drop(['PassengerId'],axis=1, inplace=True)
# Заменить значения "nan" на NULL/NAN для полей содрежащих строки
df_raw.replace('nan', nan, inplace=True)
# Заменить значения 0 на NULL/NAN для поля возраста стоимости билета
df_raw['Age'].replace(0, nan, inplace=True)

df_train = process_data(df_raw)

In [ ]:
# Исключаем из обучения поля 'Name','Ticket','Fare','Cabin'
num_cols = ['Age','SibSp','Parch','Persons_per_ticket','Price_per_person']
cat_cols = ['Survived','Pclass', 'Title', 'Sex', 'Age_category', 'Deck', 'Embarked']

df_train[['Pclass', 'Age_category']] = df_train[['Pclass', 'Age_category']].astype('category')

data = pd.concat((df_train[num_cols], pd.get_dummies(df_train[cat_cols])), axis=1)

y_train = data['Survived']
X_train = data.drop(['Survived'], axis=1)

In [ ]:
X_train

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score
#X_scaled=scaler.fit_transform(X_train)
X_scaled=X_train
for i in range(2,25):
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(X_scaled)))
    km_scores.append(-km.score(X_scaled))
    
    silhouette = silhouette_score(X_scaled,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(X_scaled,preds)
    db_score.append(db)
    print("Davies Bouldin score for number of cluster(s) {}: {}".format(i,db))
    
    v_measure = v_measure_score(y_train,preds)
    vmeasure_score.append(v_measure)
    print("V-measure score for number of cluster(s) {}: {}".format(i,v_measure))
    print("-"*100)

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,25)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,25)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,25)],y=vmeasure_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("V-measure score")
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,25)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=15)
plt.xticks([i for i in range(2,25)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,25)],y=db_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Davies-Bouldin score")
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42,n_components=2)
# используем метод fit_transform вместо fit, т.к. класс TSNE не использует метод transform
X_tsne = tsne.fit_transform(X_train)

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as pp
pp.scatter(X_tsne[:,0],X_tsne[:,1],c=y_train);
plt.xlabel("t-SNE признак 0")
plt.xlabel("t-SNE признак 1")

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score
#X_scaled=scaler.fit_transform(X_tsne)
X_scaled=X_tsne
for i in range(2,15):
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(X_scaled)))
    km_scores.append(-km.score(X_scaled))
    
    silhouette = silhouette_score(X_scaled,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(X_scaled,preds)
    db_score.append(db)
    print("Davies Bouldin score for number of cluster(s) {}: {}".format(i,db))

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,15)],y=db_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Davies-Bouldin score")
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)

plt.show()

---

In [ ]:
data.columns

In [ ]:
import umap
reducer = umap.UMAP()
# используем метод fit_transform вместо fit, т.к. класс TSNE не использует метод transform
reducer.fit(X_train)
X_umap = reducer.transform(X_train)

In [ ]:
test = X_train.iloc[0].copy()

In [ ]:
#test['Age']=1

In [ ]:
X_train.shape

In [ ]:
X_umap_test = reducer.transform(np.array(test).reshape(1, -1))

In [ ]:
X_umap_test

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as pp
pp.scatter(X_umap[:,0],X_umap[:,1],c=y_train);
pp.scatter(X_umap_test[:,0],X_umap_test[:,1],s = 250, c='blue', marker='o');

plt.xlabel("UMAP признак 0")
plt.xlabel("UMAP признак 1")

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score
X_scaled=scaler.fit_transform(X_umap)
for i in range(2,15):
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(X_scaled)))
    km_scores.append(-km.score(X_scaled))
    
    silhouette = silhouette_score(X_scaled,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(X_scaled,preds)
    db_score.append(db)
    print("Davies Bouldin score for number of cluster(s) {}: {}".format(i,db))

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,15)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=15)
plt.xticks([i for i in range(2,15)],fontsize=14)
plt.yticks(fontsize=15)

plt.show()

In [ ]:
plt.scatter(x=[i for i in range(2,15)],y=db_score,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Davies-Bouldin score")
plt.show()

---

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
X_tsne = tsne.fit_transform(X_train)
X_scaled_tsne=scaler.fit_transform(X_tsne)
km_tsne = KMeans(n_clusters=6, random_state=0).fit(X_scaled_tsne)

In [ ]:
km = KMeans(n_clusters=6, random_state=0).fit(X_scaled)

In [ ]:
import mglearn
reducer.fit(X_train)
X_umap = reducer.transform(X_train)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
km_tsne = KMeans(n_clusters=6, random_state=0).fit(X_scaled)
assignments = km_tsne.labels_
mglearn.discrete_scatter(X_scaled_tsne[:, 0], X_scaled_tsne[:, 1], assignments, ax=axes[0])

km_umap = KMeans(n_clusters=6, random_state=0).fit(X_umap)
assignments = km_umap.labels_
mglearn.discrete_scatter(X_umap[:, 0], X_umap[:, 1], assignments, ax=axes[1]);

In [ ]:
redicer=[]
for i in km_umap.cluster_centers_:
    redicer.append(reducer.inverse_transform(i.reshape(1,-1))[0])


In [ ]:
df = pd.DataFrame(np.array(redicer), columns = X_train.columns)

In [ ]:
df

---